In [ ]:
# To use this script in Google Colab, we first need to install the dependencies

import vizdoom as vzd
import gymnasium as gym
from gymnasium import spaces
import numpy as np
from stable_baselines3 import PPO

class DoomEnv(gym.Env):
    def __init__(self):
        super(DoomEnv, self).__init__()
        self.game = initialize_doom()

        # Define the action and observation space
        self.action_space = spaces.Discrete(len(self.game.get_available_buttons()))
        screen_shape = (self.game.get_screen_height(), self.game.get_screen_width(), 3)
        self.observation_space = spaces.Box(low=0, high=255, shape=screen_shape, dtype=np.uint8)

    def reset(self, seed=None, options=None):
        # Set the seed if provided
        if seed is not None:
            self.np_random, _ = gym.utils.seeding.np_random(seed)

        self.game.new_episode()
        state = self.game.get_state().screen_buffer
        return state, {}

    def step(self, action):
        actions = [0] * self.action_space.n
        actions[action] = 1
        reward = self.game.make_action(actions)
        done = self.game.is_episode_finished()
        state = self.game.get_state().screen_buffer if not done else np.zeros(self.observation_space.shape, dtype=np.uint8)
        return state, reward, done, False, {}

    def render(self, mode="human"):
        pass  # Rendering is already handled by ViZDoom

    def close(self):
        self.game.close()

def initialize_doom():
    # Initialize the Doom game
    game = vzd.DoomGame()
    game.set_doom_scenario_path(vzd.scenarios_path + "/basic.wad")  # Load the basic scenario
    game.set_doom_map("map01")

    # Set rendering options
    game.set_screen_resolution(vzd.ScreenResolution.RES_640X480)
    game.set_screen_format(vzd.ScreenFormat.RGB24)

    # Configure available buttons (actions)
    game.add_available_button(vzd.Button.MOVE_LEFT)
    game.add_available_button(vzd.Button.MOVE_RIGHT)
    game.add_available_button(vzd.Button.ATTACK)

    # Configure available game variables
    game.add_available_game_variable(vzd.GameVariable.AMMO2)

    # Set the difficulty level
    game.set_doom_skill(3)

    # Enable the rendering window
    game.set_window_visible(True)  # Change to True if working locally

    # Start the game
    game.init()
    return game

if __name__ == "__main__":
    print("Creating the Doom environment...")
    env = DoomEnv()

    print("Training the PPO model for 1000 steps...")
    model = PPO("CnnPolicy", env, verbose=1)
    model.learn(total_timesteps=1_000)

    print("Using the trained model to play for 1 minute...")
    game = initialize_doom()
    game.set_window_visible(False)  # Render the game window

    game.new_episode()
    start_time = time.time()
    while time.time() - start_time < 60:
        if game.is_episode_finished():
            game.new_episode()

        state = game.get_state()
        if state is not None:
            screen_buffer = state.screen_buffer
            action, _ = model.predict(screen_buffer)
            actions = [0] * env.action_space.n
            actions[action] = 1
            game.make_action(actions)

    game.close()
    print("Game finished.")
